In [ ]:
!pip install langchain-community

In [ ]:
!pip install python-dotenv
!pip install langchain_openai   langchain_core   langchain_neo4j

In [ ]:
# NEO4J and OPENAI Connections

from dotenv import load_dotenv
import os
load_dotenv(".env")


In [5]:
import os

from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
    )

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)


Session ID: 10d7430d-9b0a-4cb7-9fc2-32b0a646e60c


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Neo4j expert having a conversation about how to create Cypher queries",
        ),
        ("human", "{input}"),
    ]
)

In [7]:
cypher_chat = prompt | llm | StrOutputParser()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

tools = [
    Tool.from_function(
        name="Cypher Support",
        description="For when you need to talk about Cypher queries.",
        func=cypher_chat.invoke,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

cypher_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while (q := input("> ")) != "exit":

    response = cypher_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )

    print(response["output"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


> hi, i need a cypher query, to get all users who have more than 5 transactions.
To get all users who have more than 5 transactions, you can write a Cypher query like this:

```
MATCH (u:User)-[:MADE]->(t:Transaction)
WITH u, COUNT(t) AS transactionCount
WHERE transactionCount > 5
RETURN u
```

In this query:
- We first match the `User` nodes and their outgoing `MADE` relationships to `Transaction` nodes.
- We then use the `WITH` clause to group by the `User` nodes and calculate the count of transactions for each user.
- Finally, we filter for users who have more than 5 transactions and return those users.
> exit
